In [1]:
import os
os.chdir('../')
current_path = os.getcwd()


In [2]:
import sympy as sp 
import sympycalcs as spc
import sympy.physics.units as unit
from sympy.abc import *

sp.init_printing(use_latex='mathjax', latex_mode='equation')

# Beispiel: Kragarm mit 2 Punktmassen
{{< pagebreak >}}
## Aufgabenstellung

Das in @fig-kragarm_2_punkte dargestellte System stellt einen Kragarm mit verteilter Masse und 2 Punktmassen dar. Eine mögliche Formfunktion ist rechts daneben gezeigt.

![Kragarm mit verteilter Masse und zwei Punktmassen](bilder/aufgabe_rayleigh_2_massen.svg){#fig-kragarm_2_punkte}

Gesucht:

- Grundfrequenz (1. Eigenfrequenz $\omega_n$) des Systems, berechnet mit dem Rayleigh-Quotienten.

Gegeben:

- Randbedingungen für den Spezialfall: $m_{const} = 0 \text{ und } m_1 = m_2 = m$
- Formfunktion:
$$ \Psi(x) = 1 - \cos(\frac{\pi x}{2L})$$


In [3]:
L = sp.symbols('L', positive=True)
m_1, m_2, m_star, k_star, omega_1 = sp.symbols('m_1, m_2, m_star, k_star omega_1')

Psi_x = sp.Function('Psi')(x)
f = sp.Function('f')(x,t)

In [4]:
params={
    m:0,
    m_1:m,
    m_2:m
}
spc.dict_to_table(params)

|  Parameter  | ​  |
|---|---|
| $m = 0$ | $m_{1} = m$ |
| $m_{2} = m$ | ​  |


{{< pagebreak >}}
## Musterlösung

### Formfunktion

In [5]:
eq_Psi_x = sp.Eq(Psi_x, 1 - sp.cos(sp.pi*x/(2*L)))
eq_Psi_x

              ⎛π⋅x⎞
Ψ(x) = 1 - cos⎜───⎟
              ⎝2⋅L⎠

### Allgemeine Bewegungsgleichung

Mithilfe der in der Vorlesung hergeleiteten Bewegungsgleichung kann anhand der Formfunktion $\Psi$ die erste Eigenkreisfrequenz ermittelt werden. Der Rayleigh-Quotient ist eine Energiebetrachtung. Er setzt die potentielle, maximale Energie $E_{pot,max}$ zur kinetischen, maximalen Energie $E_{kin,max}$ ins Verhältnis. Daraus lässt sich die Kreisfrequenz $\omega_n$ herauslösen.


In [6]:
eq_m_star = sp.Eq(m_star,sp.Integral(m*Psi_x**2,(x, 0,L))+m_1 * Psi_x.subs(x,L)**2 + m_2 *Psi_x.subs(x,L/2)**2)
eq_k_star = sp.Eq(k_star,sp.Integral(E*I*sp.Derivative(Psi_x,x,2)**2, (x,0,L)))
display(eq_m_star, eq_k_star)

                              L           
                              ⌠           
            2          2⎛L⎞   ⎮    2      
mₛₜₐᵣ = m₁⋅Ψ (L) + m₂⋅Ψ ⎜─⎟ + ⎮ m⋅Ψ (x) dx
                        ⎝2⎠   ⌡           
                              0           

        L                    
        ⌠                    
        ⎮                2   
        ⎮     ⎛  2      ⎞    
        ⎮     ⎜ d       ⎟    
kₛₜₐᵣ = ⎮ E⋅I⋅⎜───(Ψ(x))⎟  dx
        ⎮     ⎜  2      ⎟    
        ⎮     ⎝dx       ⎠    
        ⌡                    
        0                    

In [7]:
eq_bewegung_allg = sp.Eq(f, sp.Derivative(u,x,2)*m_star+u*k_star)
eq_bewegung_allg

                            2   
                           d    
f(x, t) = kₛₜₐᵣ⋅u + mₛₜₐᵣ⋅───(u)
                            2   
                          dx    

In [8]:
spc.eq_subs(eq_bewegung_allg, eq_m_star, eq_k_star)

            L                                                                 
            ⌠                                                                 
            ⎮                2      ⎛                      L           ⎞      
            ⎮     ⎛  2      ⎞       ⎜                      ⌠           ⎟   2  
            ⎮     ⎜ d       ⎟       ⎜    2          2⎛L⎞   ⎮    2      ⎟  d   
f(x, t) = u⋅⎮ E⋅I⋅⎜───(Ψ(x))⎟  dx + ⎜m₁⋅Ψ (L) + m₂⋅Ψ ⎜─⎟ + ⎮ m⋅Ψ (x) dx⎟⋅───(u
            ⎮     ⎜  2      ⎟       ⎜                ⎝2⎠   ⌡           ⎟   2  
            ⎮     ⎝dx       ⎠       ⎝                      0           ⎠ dx   
            ⌡                                                                 
            0                                                                 

 
 
 
 
 
)
 
 
 
 

Substituiert mit Massen- und Steifigkeitsvariable:

### Eigenkreisfrequenz

Aus der Bewegungsgleichung kann die Eigenkreisfrequenz ermittelt werden:

In [9]:
eq_eigenkreisfreq = sp.Eq(omega_1, sp.sqrt(k_star/ m_star))
eq_eigenkreisfreq

         _______
        ╱ kₛₜₐᵣ 
ω₁ =   ╱  ───── 
     ╲╱   mₛₜₐᵣ 

#### Bestimmung der Masse

Die Masse kann mittels der Lösung des Integrals bestimmt werden. Dabei sind die Punktmassen mittels der entsprechenden Deformation an den Stellen $L$ und $\frac{L}{2}$ zu berücksichtigen, sowie die verteilte Masse über die gesamte Länge.


In [10]:
eq_Psi_x.subs(x,L)

Ψ(L) = 1

In [11]:
eq_Psi_x.subs(x,L/2)

 ⎛L⎞       √2
Ψ⎜─⎟ = 1 - ──
 ⎝2⎠       2 

In [12]:
sp.Eq(eq_Psi_x.lhs**2, eq_Psi_x.rhs**2)

                      2
 2      ⎛       ⎛π⋅x⎞⎞ 
Ψ (x) = ⎜1 - cos⎜───⎟⎟ 
        ⎝       ⎝2⋅L⎠⎠ 

In [13]:
spc.eq_subs(eq_m_star,eq_Psi_x, eq_Psi_x.subs(x,L), eq_Psi_x.subs(x,L/2))

                            L                     
                            ⌠                     
                        2   ⎮                 2   
                ⎛    √2⎞    ⎮   ⎛       ⎛π⋅x⎞⎞    
mₛₜₐᵣ = m₁ + m₂⋅⎜1 - ──⎟  + ⎮ m⋅⎜1 - cos⎜───⎟⎟  dx
                ⎝    2 ⎠    ⎮   ⎝       ⎝2⋅L⎠⎠    
                            ⌡                     
                            0                     

In [14]:
spc.eq_subs(eq_m_star,eq_Psi_x, eq_Psi_x.subs(x,L), eq_Psi_x.subs(x,L/2)).doit()

                                          2
          ⎛  4⋅L   3⋅L⎞           ⎛    √2⎞ 
mₛₜₐᵣ = m⋅⎜- ─── + ───⎟ + m₁ + m₂⋅⎜1 - ──⎟ 
          ⎝   π     2 ⎠           ⎝    2 ⎠ 

#### Bestimmung der Steifigkeit

Die Steifigkeit in kann mittels der Lösung des Integrals in bestimmt werden. 
Zur Ermittlung der Steifigkeit $k^\star$ muss zuerst der Ansatz zweimal nach $x$ abgeleitet werden.

In [15]:
display(eq_Psi_x,sp.Eq(sp.diff(Psi_x, x, 1),sp.diff(eq_Psi_x.rhs,x,1).doit()),sp.Eq(sp.diff(Psi_x, x, 2),sp.diff(eq_Psi_x.rhs,x,2)))

              ⎛π⋅x⎞
Ψ(x) = 1 - cos⎜───⎟
              ⎝2⋅L⎠

                ⎛π⋅x⎞
           π⋅sin⎜───⎟
d               ⎝2⋅L⎠
──(Ψ(x)) = ──────────
dx            2⋅L    

             2    ⎛π⋅x⎞
  2         π ⋅cos⎜───⎟
 d                ⎝2⋅L⎠
───(Ψ(x)) = ───────────
  2                2   
dx              4⋅L    

Danach bedingt es lediglich das einsetzen:

In [16]:
spc.eq_subs(eq_k_star, eq_Psi_x).doit()

         4    
        π ⋅E⋅I
kₛₜₐᵣ = ──────
            3 
        32⋅L  

In [17]:
display(spc.eq_subs(eq_k_star, eq_Psi_x),
       spc.eq_subs(eq_k_star,eq_Psi_x).doit())

        L                            
        ⌠                            
        ⎮                        2   
        ⎮     ⎛  2              ⎞    
        ⎮     ⎜ ∂ ⎛       ⎛π⋅x⎞⎞⎟    
kₛₜₐᵣ = ⎮ E⋅I⋅⎜───⎜1 - cos⎜───⎟⎟⎟  dx
        ⎮     ⎜  2⎝       ⎝2⋅L⎠⎠⎟    
        ⎮     ⎝∂x               ⎠    
        ⌡                            
        0                            

         4    
        π ⋅E⋅I
kₛₜₐᵣ = ──────
            3 
        32⋅L  

#### Grundfrequenz
Letztlich kann die Grundfrequenz bestimmt werden:

In [18]:
omega_1_sub = spc.eq_subs(eq_eigenkreisfreq,eq_k_star, eq_m_star,eq_Psi_x, eq_Psi_x.subs(x,L), eq_Psi_x.subs(x,L/2)).simplify()

omega_1_sub

                 __________________________________________
         2      ╱                   E⋅I                    
     √2⋅π ⋅    ╱  ──────────────────────────────────────── 
              ╱       ⎛  4⋅L   3⋅L⎞                      2 
             ╱    4⋅m⋅⎜- ─── + ───⎟ + 4⋅m₁ + m₂⋅(-2 + √2)  
           ╲╱         ⎝   π     2 ⎠                        
ω₁ = ──────────────────────────────────────────────────────
                                3/2                        
                             4⋅L                           

#### Auswertung des Spezialfalls

Mit Hilfe der Randbedingungen für den Spezialfall aus der Aufgabenstellung resultiert die Grundfrequenz zu:

In [19]:
omega_1_sub.subs(params).simplify().evalf(3)

                     0.5
           -1.5 ⎛E⋅I⎞   
ω₁ = 1.67⋅L    ⋅⎜───⎟   
                ⎝ m ⎠   